<a href="https://colab.research.google.com/github/YugyeongJo/study_databases/blob/main/docs/quests/CDAs/CDA_mixed_unNormal02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
### 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab의 경우 Runtime > Restart session 필요

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [36]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

## load dataset

In [37]:
import pandas as pd
import seaborn as sns
import scipy.stats as stats
df_saleshistory = pd.read_csv('/content/fruitstore_saleshistory.csv')
df_saleshistory[:5]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05
2,2018-01-02,화,1,1,1506656256,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,수지구,풍덕천동,여,36.0,30대이하,7083,매장C,NaN,10:05
3,2018-01-02,화,1,1,1023108864,정상회원,반찬,두부/유부,두부,연두부(100g),1.0,수지구,풍덕천동,여,36.0,30대이하,766,매장C,NaN,10:08
4,2018-01-02,화,1,1,1476143616,정상회원,간식,빵,"식빵, 식사대용",아침빵(240g/8개)/매장,1.0,수지구,풍덕천동,여,34.0,30대이하,4403,매장C,NaN,10:09


## 범주형/연속형

### 이분산 : 요일별 고객의 구매 금액 분산이 이분산을 보이는지?



#### 결측치, 이상치 확인

In [38]:
df_saleshistory.isnull().sum()
# 요일과 구매 금액 column은 결측치 x

공급일자             0
요일               0
공급월              0
공급주차             0
회원번호             0
조합원상태            0
물품대분류            0
물품중분류            0
물품소분류            0
물품명              0
구매수량             0
주소-구             0
주소-동             0
성별               5
연령              33
연령대             33
구매금액             0
구매매장             0
반품_원거래일자    694337
구매시각             0
dtype: int64

In [39]:
df_saleshistory['요일'].unique()

array(['화', '수', '목', '금', '토', '일', '월'], dtype=object)

In [40]:
df_saleshistory['구매금액'].unique()
df_saleshistory['구매금액']

0         22207
1          4977
2          7083
3           766
4          4403
          ...  
705566     9189
705567     5169
705568     7466
705569     3542
705570     1914
Name: 구매금액, Length: 705571, dtype: int64

#### 요일별 구매금액 dataset 설정

In [41]:
df_saleshistory_week_amount = df_saleshistory[['요일', '구매금액']]

In [42]:
# df_saleshistory_pivot = pd.pivot_table(data=df_saleshistory, index='index', values=['구매금액', '요일'])
# df_saleshistory_pivot

1) 정규화 검증

In [43]:
df_saleshistory["구매금액"].count()
# 5000개 이상 Anderson test

705571

In [44]:
# stats.anderson을 사용한 정규 분포 검정
result = stats.anderson(df_saleshistory['구매금액'])
print('Test Statistic: %.3f' % result.statistic)
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < cv:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따름 (검정 통계량 %.3f < 임계값 %.3f)' % (sl, result.statistic, cv))
    else:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 %.3f >= 임계값 %.3f)' % (sl, result.statistic, cv))
# 비정규분포 / 범주형 3집단 이상

Test Statistic: 80887.237
15.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.576)
10.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.656)
5.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.787)
2.5% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.918)
1.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 1.092)


#### 2) 범주형 3집단 Kruskal Test

##### 분석 내용 : 요일별 고객의 구매 금액 분산이 이분산을 보이는지?
- 귀무 가설 : 요일별 고객의 구매 금액은 차이가 없다.
- 대립 가설 : 요일별 고객의 구매 금액은 차이가 있다.

In [49]:
# 요일 범주 2개 이상(3집단 이상)
week_list = df_saleshistory['요일'].value_counts().index.to_list()
week_list

['월', '화', '금', '수', '목', '토', '일']

In [50]:
series_list = list()
for week in week_list:
  condition = f'요일 == "{week}"'
  series_frequency = df_saleshistory_week_amount.query(condition)['구매금액'] # condition에 따른 amount series 작성
  series_list.append(series_frequency)
series_list

[24177      2489
 24178      7466
 24179      7275
 24180      5743
 24181      7083
           ...  
 684182    50732
 684183     2680
 684184    10338
 684185    11104
 684186    23739
 Name: 구매금액, Length: 123802, dtype: int64,
 0         22207
 1          4977
 2          7083
 3           766
 4          4403
           ...  
 687767     6700
 687768     6318
 687769    11486
 687770      766
 687771     4977
 Name: 구매금액, Length: 111196, dtype: int64,
 13794      3733
 13795      4499
 13796      3063
 13797      3829
 13798     14932
           ...  
 700966     3733
 700967    11678
 700968     8423
 700969     5743
 700970     2489
 Name: 구매금액, Length: 109871, dtype: int64,
 5067       7275
 5068       4212
 5069       8806
 5070       2872
 5071       2489
           ...  
 692621    22207
 692622     9955
 692623     4977
 692624    15124
 692625    26610
 Name: 구매금액, Length: 104990, dtype: int64,
 9538       5169
 9539      13784
 9540      18953
 9541       3637
 9542       

In [51]:
stats.kruskal(*series_list)

KruskalResult(statistic=105.68763451021304, pvalue=1.6278456577231045e-20)

In [48]:
round(1.6278456577231045e-2, 6)

0.016278

In [48]:
# pvalue가 0.01(0.05보다 작다) = 대립가설 채택 = 요일별 고객의 구매 금액은 차이가 있다.

## 분석 결과 : 요일별(범주형) 고객의 구매 금액(연속형)은 차이가 있다.
- 통계 결론 : 대립 가설 성립
- 사실 결론 : 요일별 구매 금액의 차이를 고려하여 요일별 주력 판매상품을 검토해볼 필요가 있다.